In [ ]:
config = {
    "pretrained_model": "bert-base-cased",
    "tokenizer": "bert-base-cased",
    "max_seq_length": 512,
    "batch_size": 10,
    "lr": 2e-5,
    "epochs": 10,
    "device": "cuda",
    "gpu_ids": "2",
    "seed": 2020,
    "fp16": False,
    "loss_scale": 0,
    "gradient_accumulation_steps":1,
    "warmup_proportion": 0.1,
    "gradient_accumulation_steps": 1,
    "num_labels": 4,
    "is_multilabel": False,
    "valid_metric": "macro_f1",
    "model_save_dir": "../checkpoints/bert_cased_512_biocaster_4cate_20200903/",
    "result_file":"../results/bert_biocaster_202009091.csv",
    "patience": 4,
}

In [ ]:
import sys
sys.path.append("../")
from beta_nlp.utils.data_util import biocaster2df,split_data
data_file = "/home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml"
data_df = biocaster2df(data_file)
train_set,dev_set,test_set = split_data(data_df,config["seed"])

In [ ]:
def train():
    ata_df = biocaster2df(data_file)
    train_set,dev_set,test_set = split_data(data_df,config["seed"])
    from beta_nlp.models.bert_cls import BertModel
    from beta_nlp.utils.common import save_to_csv
    cls = BertModel(config)
    cls.train(train_set,dev_set)
    result = cls.test(test_set)
    result.update(config)
    save_to_csv(result, config["result_file"])
    del cls

In [ ]:
config["model"] = "BERT"
pretrained_model_opt= ["bert-base-uncased"]
max_seq_length_opt = [256,512]
lr_opt = [5e-5, 3e-5, 2e-5]
for i in range(10):
    for lr in lr_opt:
        for max_seq_length in max_seq_length_opt:
            for pretrained_model in pretrained_model_opt:
                if pretrained_model== "bert-base-cased" or max_seq_length>500:
                    config["batch_size"] = 8
                else:
                    config["batch_size"] = 16
                config["lr"]=lr
                config["max_seq_length"] = max_seq_length
                config["pretrained_model"] = pretrained_model
                config["model_save_dir"] = "../checkpoints/"+"bert_"+pretrained_model+str(max_seq_length)+"_"+str(lr)
                train()